In [ ]:
# pip install imodels

In [ ]:
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, mean_absolute_error, accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error
from datetime import datetime
from datetime import timedelta
from imodels.importance import RandomForestPlusRegressor, RandomForestPlusClassifier, \
    RidgeRegressorPPM, LassoRegressorPPM, IdentityTransformer
from imodels.importance.rf_plus import _fast_r2_score

In [2]:
import multiprocessing
multiprocessing.cpu_count()

12

In [2]:
data = pd.read_csv('PRSA_data_2010.1.1-2014.12.31_1.csv', header=0)

In [17]:
data
data = data.dropna()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
24,25,2010,1,2,0,129.0,-16,-4.0,1020.0,SE,1.79,0,0
25,26,2010,1,2,1,148.0,-15,-4.0,1020.0,SE,2.68,0,0
26,27,2010,1,2,2,159.0,-11,-5.0,1021.0,SE,3.57,0,0
27,28,2010,1,2,3,181.0,-7,-5.0,1022.0,SE,5.36,1,0
28,29,2010,1,2,4,138.0,-7,-5.0,1022.0,SE,6.25,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,43820,2014,12,31,19,8.0,-23,-2.0,1034.0,NW,231.97,0,0
43820,43821,2014,12,31,20,10.0,-22,-3.0,1034.0,NW,237.78,0,0
43821,43822,2014,12,31,21,10.0,-22,-3.0,1034.0,NW,242.70,0,0
43822,43823,2014,12,31,22,8.0,-22,-4.0,1034.0,NW,246.72,0,0


In [8]:
data = pd.get_dummies(data)

In [9]:
data

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_cv
24,25,2010,1,2,0,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0
25,26,2010,1,2,1,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0
26,27,2010,1,2,2,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0
27,28,2010,1,2,3,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0
28,29,2010,1,2,4,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,43820,2014,12,31,19,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0
43820,43821,2014,12,31,20,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0
43821,43822,2014,12,31,21,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0
43822,43823,2014,12,31,22,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0


In [10]:
train_data_by_month, test_data_by_month = {}, {}
train_data_by_year, test_data_by_year = {}, {}

# Splitting data for each month (month-year combination) separately
for year in data['year'].unique():
    for month in data['month'].unique():
        month_year_data = data[(data['year'] == year) & (data['month'] == month)]
        train_data_by_month[(year, month)], test_data_by_month[(year, month)] = train_test_split(month_year_data,
                                                                                                 test_size=0.2, random_state=12345)

# Combine the monthly splits into yearly splits
for year in data['year'].unique():
    yearly_train_data = pd.concat([train_data_by_month[(year, month)] for month in data['month'].unique()])
    yearly_test_data = pd.concat([test_data_by_month[(year, month)] for month in data['month'].unique()])
    train_data_by_year[year] = yearly_train_data
    test_data_by_year[year] = yearly_test_data

# Combine all monthly splits into a single dataset for full range modeling
train_data_full = pd.concat(train_data_by_month.values())
test_data_full = pd.concat(test_data_by_month.values())

In [11]:
train_data_by_month.items()

dict_items([((2010, 1),       No  year  month  day  hour  pm2.5  DEWP  TEMP    PRES     Iws  Is  Ir  \
620  621  2010      1   26    20  274.0   -11  -5.0  1020.0    0.89   0   0   
367  368  2010      1   16     7  114.0   -15 -11.0  1034.0    4.47   0   0   
627  628  2010      1   27     3  213.0   -11  -4.0  1016.0   10.29   0   0   
90    91  2010      1    4    18   28.0   -23 -11.0  1032.0  165.37   0   0   
178  179  2010      1    8    10  135.0   -17 -10.0  1030.0    0.89   0   0   
..   ...   ...    ...  ...   ...    ...   ...   ...     ...     ...  ..  ..   
444  445  2010      1   19    12  301.0    -5   1.0  1020.0    0.89   0   0   
153  154  2010      1    7     9   27.0   -23 -12.0  1036.0    8.94   0   0   
309  310  2010      1   13    21   95.0   -19 -13.0  1027.0    0.89   0   0   
509  510  2010      1   22     5   15.0   -20  -7.0  1035.0   48.27   0   0   
506  507  2010      1   22     2   26.0   -21  -5.0  1037.0   27.26   0   0   

     cbwd_NE  cbwd_NW  cbwd

In [12]:
test_data_by_month.keys()

dict_keys([(2010, 1), (2010, 2), (2010, 3), (2010, 4), (2010, 5), (2010, 6), (2010, 7), (2010, 8), (2010, 9), (2010, 10), (2010, 11), (2010, 12), (2011, 1), (2011, 2), (2011, 3), (2011, 4), (2011, 5), (2011, 6), (2011, 7), (2011, 8), (2011, 9), (2011, 10), (2011, 11), (2011, 12), (2012, 1), (2012, 2), (2012, 3), (2012, 4), (2012, 5), (2012, 6), (2012, 7), (2012, 8), (2012, 9), (2012, 10), (2012, 11), (2012, 12), (2013, 1), (2013, 2), (2013, 3), (2013, 4), (2013, 5), (2013, 6), (2013, 7), (2013, 8), (2013, 9), (2013, 10), (2013, 11), (2013, 12), (2014, 1), (2014, 2), (2014, 3), (2014, 4), (2014, 5), (2014, 6), (2014, 7), (2014, 8), (2014, 9), (2014, 10), (2014, 11), (2014, 12)])

In [13]:
rf_regressor = RandomForestRegressor(n_estimators=50,min_samples_leaf=3,
                                     random_state=331,n_jobs=-1)
rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor)


In [14]:
training_results = {}
testing_results = {}
prediction_vectors = pd.DataFrame()
col_drop = ['pm2.5', 'No', 'month']
for (year, month), train_data in train_data_by_month.items():
    test_data = test_data_by_month[(year, month)]


    X_train = train_data.drop(col_drop, axis=1)
    y_train = train_data['pm2.5'].to_numpy()
    X_test = test_data.drop(col_drop, axis=1)
    y_test = test_data['pm2.5'].to_numpy()
    
    # Initialize RF+
    rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor)
    rf_plus_model.fit(X_train, y_train)
    
    # predictions
    train_pred = rf_plus_model.predict(X_train)
    test_pred = rf_plus_model.predict(X_test)

    # Calculate RMSE
    train_rmse = mean_squared_error(y_train, train_pred, squared=False)
    test_rmse = mean_squared_error(y_test, test_pred, squared=False)
    training_results[(year, month)] = train_rmse
    testing_results[(year, month)] = test_rmse

    # store in dataframe
    train_df = pd.DataFrame({'Year': year, 'Month': month, 'Type': 'Train', 'Y_True': y_train, 'Y_Pred': train_pred})
    test_df = pd.DataFrame({'Year': year, 'Month': month, 'Type': 'Test', 'Y_True': y_test, 'Y_Pred': test_pred})

    # Concatenate with the main DataFrame
    prediction_vectors = pd.concat([prediction_vectors, train_df, test_df])

    # Print results for each month
    print(f"Year: {year}, Month: {month}, Training RMSE: {train_rmse}, Testing RMSE: {test_rmse}")

# Results include: training_results, testing_results, and prediction_vectors

Year: 2010, Month: 1, Training RMSE: 23.424707693696778, Testing RMSE: 33.12501920908587
Year: 2010, Month: 2, Training RMSE: 38.19839122688131, Testing RMSE: 32.28752616064923
Year: 2010, Month: 3, Training RMSE: 37.9394502114286, Testing RMSE: 28.094626162462987
Year: 2010, Month: 4, Training RMSE: 14.375163682205523, Testing RMSE: 17.58771969301951
Year: 2010, Month: 5, Training RMSE: 17.651176321054873, Testing RMSE: 31.997085252466118
Year: 2010, Month: 6, Training RMSE: 17.41254568148024, Testing RMSE: 26.651151363765887
Year: 2010, Month: 7, Training RMSE: 22.30514150697771, Testing RMSE: 35.26171357348102
Year: 2010, Month: 8, Training RMSE: 21.45941002453309, Testing RMSE: 30.702619434067007
Year: 2010, Month: 9, Training RMSE: 24.520077316650685, Testing RMSE: 37.76067950648548
Year: 2010, Month: 10, Training RMSE: 22.722853305048787, Testing RMSE: 29.301741131613642
Year: 2010, Month: 11, Training RMSE: 34.55409011383064, Testing RMSE: 48.54930327537121
Year: 2010, Month: 12

In [15]:
prediction_vectors

,Year,Month,Type,Y_True,Y_Pred
0,2010,1,Train,274.0,248.268393
1,2010,1,Train,114.0,130.960433
2,2010,1,Train,213.0,196.823501
3,2010,1,Train,28.0,25.503752
4,2010,1,Train,135.0,114.211887
...,...,...,...,...,...
139,2014,12,Test,11.0,22.419623
140,2014,12,Test,9.0,10.262622
141,2014,12,Test,119.0,112.060237
142,2014,12,Test,71.0,62.938050


In [44]:
grouped = prediction_vectors.groupby(['Year', 'Type'])

# Function to calculate RMSE
def calculate_rmse(group):
    return np.sqrt(mean_squared_error(group['Y_True'], group['Y_Pred']))

# Calculate RMSE for each group
yearly_rmse = grouped.apply(calculate_rmse).unstack()
print('annual error by monthly model:')
print(yearly_rmse)

annual error by monthly model:
Type       Test      Train
Year                      
2010  32.650022  26.209507
2011  32.277414  22.393528
2012  30.790477  22.635599
2013  35.374359  25.372149
2014  31.304605  21.448386


In [41]:
training_results.values()

dict_values([23.424707693696778, 38.19839122688131, 37.9394502114286, 14.375163682205523, 17.651176321054873, 17.41254568148024, 22.30514150697771, 21.45941002453309, 24.520077316650685, 22.722853305048787, 34.55409011383064, 26.23554101098068, 16.35046360923468, 31.538973541367717, 17.24905884582534, 23.29102075866275, 15.845548223829182, 25.64387376278841, 17.947213859531843, 13.696367154956649, 16.733001179702953, 27.512908822096847, 21.712085387293566, 30.84376820669694, 46.105127661290176, 21.38725020344679, 19.13697032207496, 16.686305984281667, 19.138357354694598, 21.108171643005928, 14.349525573497726, 12.747868817794764, 12.3665470705954, 25.179040903174286, 18.739473515065534, 25.634454039047384, 49.15600258373044, 32.828161034420845, 26.20066128704172, 14.989885489514455, 17.916344791649877, 23.44296569025228, 14.212770091117385, 11.92524402531016, 16.081866684386576, 22.819942982598064, 24.52847040591284, 26.967060698092183, 32.95969552352978, 24.709297838075972, 21.7051317

In [31]:
train_data_by_year.items()

dict_items([(2010,         No  year  month  day  hour  pm2.5  DEWP  TEMP    PRES     Iws  Is  Ir  \
620    621  2010      1   26    20  274.0   -11  -5.0  1020.0    0.89   0   0   
367    368  2010      1   16     7  114.0   -15 -11.0  1034.0    4.47   0   0   
627    628  2010      1   27     3  213.0   -11  -4.0  1016.0   10.29   0   0   
90      91  2010      1    4    18   28.0   -23 -11.0  1032.0  165.37   0   0   
178    179  2010      1    8    10  135.0   -17 -10.0  1030.0    0.89   0   0   
...    ...   ...    ...  ...   ...    ...   ...   ...     ...     ...  ..  ..   
8436  8437  2010     12   18    12  206.0   -13   3.0  1012.0    1.79   0   0   
8145  8146  2010     12    6     9   15.0   -18   0.0  1035.0  193.97   0   0   
8301  8302  2010     12   12    21  140.0    -7   0.0  1023.0    2.23   0   0   
8501  8502  2010     12   21     5  367.0   -10  -5.0  1013.0    1.79   0   0   
8498  8499  2010     12   21     2  391.0   -10  -4.0  1014.0    3.57   0   0   

      cb

In [34]:
year_training_results = {}
year_testing_results = {}
year_prediction_vectors = pd.DataFrame()
col_drop = ['pm2.5', 'No', 'year']


for year, train_data in train_data_by_year.items():
    test_data = test_data_by_year[year]
    X_train = train_data.drop(col_drop, axis=1)
    y_train = train_data['pm2.5'].to_numpy()
    X_test = test_data.drop(col_drop, axis=1)
    y_test = test_data['pm2.5'].to_numpy()
    
    # Initialize RF+
    rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor)
    rf_plus_model.fit(X_train, y_train)
    
    # Make predictions and calculate metrics
    train_pred = rf_plus_model.predict(X_train)
    test_pred = rf_plus_model.predict(X_test)
    train_rmse = mean_squared_error(y_train, train_pred, squared=False)
    test_rmse = mean_squared_error(y_test, test_pred, squared=False)
    
    # Store and print results
    year_training_results[year] = train_rmse
    year_testing_results[year] = test_rmse
    train_df = pd.DataFrame({'Year': year, 'Type': 'Train', 'Y_True': y_train, 'Y_Pred': train_pred})
    test_df = pd.DataFrame({'Year': year, 'Type': 'Test', 'Y_True': y_test, 'Y_Pred': test_pred})
    year_prediction_vectors = pd.concat([year_prediction_vectors, train_df, test_df])
    print(f"Year: {year}, Training RMSE: {train_rmse}, Testing RMSE: {test_rmse}")


Year: 2010, Training RMSE: 28.161938275508252, Testing RMSE: 36.01119332101772
Year: 2011, Training RMSE: 24.589363083306257, Testing RMSE: 36.180777593499755
Year: 2012, Training RMSE: 25.260428825639117, Testing RMSE: 34.99107699110881
Year: 2013, Training RMSE: 26.649158251502943, Testing RMSE: 37.09343560863756
Year: 2014, Training RMSE: 22.720241260005963, Testing RMSE: 33.53610464003447


In [19]:
year_prediction_vectors

NameError: name 'year_prediction_vectors' is not defined

In [16]:
train_data_full

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_cv
620,621,2010,1,26,20,274.0,-11,-5.0,1020.0,0.89,0,0,1,0,0,0
367,368,2010,1,16,7,114.0,-15,-11.0,1034.0,4.47,0,0,0,1,0,0
627,628,2010,1,27,3,213.0,-11,-4.0,1016.0,10.29,0,0,0,1,0,0
90,91,2010,1,4,18,28.0,-23,-11.0,1032.0,165.37,0,0,0,1,0,0
178,179,2010,1,8,10,135.0,-17,-10.0,1030.0,0.89,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43519,43520,2014,12,19,7,66.0,-12,0.0,1023.0,22.35,0,0,0,1,0,0
43211,43212,2014,12,6,11,30.0,-20,2.0,1028.0,9.83,0,0,1,0,0,0
43384,43385,2014,12,13,16,44.0,-19,4.0,1026.0,8.04,0,0,0,0,1,0
43593,43594,2014,12,22,9,145.0,-13,-4.0,1026.0,5.37,0,0,1,0,0,0


In [17]:
test_data_full

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_cv
264,265,2010,1,12,0,21.0,-22,-15.0,1031.0,1.78,0,0,0,0,0,1
487,488,2010,1,21,7,18.0,-25,-9.0,1038.0,138.10,0,0,0,1,0,0
528,529,2010,1,23,0,49.0,-17,-4.0,1033.0,172.08,0,0,0,1,0,0
438,439,2010,1,19,6,330.0,-4,-1.0,1021.0,12.07,0,0,0,1,0,0
685,686,2010,1,29,13,12.0,-24,5.0,1022.0,50.07,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43193,43194,2014,12,5,17,11.0,-22,0.0,1025.0,1.79,0,0,1,0,0,0
43189,43190,2014,12,5,13,9.0,-23,3.0,1026.0,36.20,0,0,0,1,0,0
43221,43222,2014,12,6,21,119.0,-14,0.0,1026.0,0.89,0,0,0,0,0,1
43216,43217,2014,12,6,16,71.0,-18,2.0,1025.0,5.37,0,0,0,0,1,0


In [20]:
start = datetime.now()
col_drop = ['pm2.5', 'No']
train_data= train_data_full
test_data = test_data_full
X_train = train_data.drop(col_drop, axis=1)
y_train = train_data['pm2.5'].to_numpy()
X_test = test_data.drop(col_drop, axis=1)
y_test = test_data['pm2.5'].to_numpy()

# Initialize RF+
rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor)
rf_plus_model.fit(X_train, y_train)




end = datetime.now()
end-start

datetime.timedelta(seconds=15410, microseconds=215831)

In [21]:
# Make predictions and calculate metrics
start = datetime.now()
train_pred = rf_plus_model.predict(X_train)
test_pred = rf_plus_model.predict(X_test)
train_rmse = mean_squared_error(y_train, train_pred, squared=False)
test_rmse = mean_squared_error(y_test, test_pred, squared=False)
end = datetime.now()
end-start

datetime.timedelta(seconds=871, microseconds=306282)

In [22]:
# Store and print results
training_results = train_rmse
testing_results= test_rmse

In [ ]:
train_df = pd.DataFrame({'Year': year, 'Type': 'Train', 'Y_True': y_train, 'Y_Pred': train_pred})
test_df = pd.DataFrame({'Year': year, 'Type': 'Test', 'Y_True': y_test, 'Y_Pred': test_pred})
year_prediction_vectors = pd.concat([year_prediction_vectors, train_df, test_df])
print(f"Year: {year}, Training RMSE: {train_rmse}, Testing RMSE: {test_rmse}")

In [23]:
training_results

27.71042372510132

In [24]:
testing_results

38.62706504633146

In [25]:
import pickle

# Assume 'model' is your trained model
with open('RFPlus_full.pkl', 'wb') as file:
    pickle.dump(rf_plus_model, file)

PicklingError: Can't pickle <function _GlmPPM.<lambda> at 0x000001A3D71E2680>: attribute lookup _GlmPPM.<lambda> on imodels.importance.ppms failed